In [1]:
#https://codelabs.developers.google.com/codelabs/cloud-tensorflow-mnist/
#https://www.youtube.com/watch?v=vq2nnJ4g6N0&t=911s

import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
tf.set_random_seed(0)

mnist = read_data_sets("data", one_hot=True, reshape=False, validation_size=0)

Extracting data\train-images-idx3-ubyte.gz
Extracting data\train-labels-idx1-ubyte.gz
Extracting data\t10k-images-idx3-ubyte.gz
Extracting data\t10k-labels-idx1-ubyte.gz


In [2]:
mnist.train.images.shape #here we have gray images

(60000, 28, 28, 1)

In [3]:
mnist.test.images.shape

(10000, 28, 28, 1)

In [4]:
mnist.validation.images.shape #so no validation set here

(0, 28, 28, 1)

In [5]:
type(mnist.train.images) #so numpy array it should be simple

numpy.ndarray

### Applying simple linear regression

In [6]:
X = tf.placeholder(tf.float32,[None,28,28,1]) #we donot know here how many training images are we gonna give
W = tf.Variable(tf.zeros([784,10])) #variables are that we want to change
b = tf.Variable(tf.zeros([10]))

init = tf.global_variables_initializer() #operation to initialize the variable 

In [7]:
#model 

Y = tf.nn.softmax(tf.matmul(tf.reshape(X,[-1,784]),W) + b)

Y_ = tf.placeholder(tf.float32,[None,10])  #correct labels

In [8]:
#loss function
cross_entropy = - tf.reduce_sum(Y_ * tf.log(Y)) # + 0.5 * tf.reduce_sum(tf.multiply(W,W)) #Adding regularization here

In [9]:
graph = tf.get_default_graph()

In [10]:
graph.get_operations()

[<tf.Operation 'Placeholder' type=Placeholder>,
 <tf.Operation 'zeros' type=Const>,
 <tf.Operation 'Variable' type=VariableV2>,
 <tf.Operation 'Variable/Assign' type=Assign>,
 <tf.Operation 'Variable/read' type=Identity>,
 <tf.Operation 'zeros_1' type=Const>,
 <tf.Operation 'Variable_1' type=VariableV2>,
 <tf.Operation 'Variable_1/Assign' type=Assign>,
 <tf.Operation 'Variable_1/read' type=Identity>,
 <tf.Operation 'init' type=NoOp>,
 <tf.Operation 'Reshape/shape' type=Const>,
 <tf.Operation 'Reshape' type=Reshape>,
 <tf.Operation 'MatMul' type=MatMul>,
 <tf.Operation 'add' type=Add>,
 <tf.Operation 'Softmax' type=Softmax>,
 <tf.Operation 'Placeholder_1' type=Placeholder>,
 <tf.Operation 'Log' type=Log>,
 <tf.Operation 'mul' type=Mul>,
 <tf.Operation 'Const' type=Const>,
 <tf.Operation 'Sum' type=Sum>,
 <tf.Operation 'Neg' type=Neg>]

In [11]:
# % of correct answers found in the batch 
is_correct = tf.equal(tf.argmax(Y,1),tf.argmax(Y_,1)) #it will compute bool that is why cast is needed in the statement below
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))

In [12]:
optimizer = tf.train.GradientDescentOptimizer(0.003)

In [13]:
train_step = optimizer.minimize(cross_entropy)

In [14]:
#no execution so far
#graph.get_operations()

### Differed execution 

In [15]:
train_step

<tf.Operation 'GradientDescent' type=NoOp>

In [16]:
optimizer

### Now to the execution------------------

In [17]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [18]:
sess = tf.Session(config = config)
sess.run(init)

In [19]:
#just testing
print("%f"%0.5)

0.500000


In [20]:
for i in range(10001):
    batch_X,batch_Y = mnist.train.next_batch(100)
    train_data = {X : batch_X, Y_ : batch_Y}
    
    #training the system
    sess.run(train_step , feed_dict = train_data)
    
    #success
    a,c = sess.run([accuracy , cross_entropy],feed_dict= train_data)
    if i % 1000 == 0:
        print()
        print("Iteration number %d Training accuracy %f cross entropy loss %f"%(i,a,c))
        #success on test data
        
        test_data = {X : mnist.test.images,Y_ : mnist.test.labels}
        a,c = sess.run([accuracy,cross_entropy],feed_dict = test_data)
        
        print("Iteration number %d Testing accuracy %f cross entropy loss %f"%(i,a,c))



Iteration number 0 Training accuracy 0.440000 cross entropy loss 183.742615
Iteration number 0 Testing accuracy 0.241000 cross entropy loss 20600.937500

Iteration number 1000 Training accuracy 0.900000 cross entropy loss 34.714710
Iteration number 1000 Testing accuracy 0.917600 cross entropy loss 2956.397949

Iteration number 2000 Training accuracy 0.970000 cross entropy loss 15.753893
Iteration number 2000 Testing accuracy 0.919400 cross entropy loss 2864.634766

Iteration number 3000 Training accuracy 0.960000 cross entropy loss 21.403524
Iteration number 3000 Testing accuracy 0.922600 cross entropy loss 2760.903564

Iteration number 4000 Training accuracy 0.940000 cross entropy loss 25.103807
Iteration number 4000 Testing accuracy 0.920700 cross entropy loss 2737.466797

Iteration number 5000 Training accuracy 0.920000 cross entropy loss 22.589405
Iteration number 5000 Testing accuracy 0.923800 cross entropy loss 2719.710205

Iteration number 6000 Training accuracy 0.920000 cross 

### Adding layers 

In [37]:
W1 = tf.Variable(tf.truncated_normal([28*28,200],stddev= 0.1))
B1 = tf.Variable(tf.zeros([200]))

W2 = tf.Variable(tf.truncated_normal([200,10],stddev= 0.1))
B2 = tf.Variable(tf.truncated_normal([10]))

In [38]:
Y1 =  tf.nn.sigmoid(tf.matmul(tf.reshape(X,[-1,784]),W1) + B1)
Y_out = tf.nn.softmax(tf.matmul(Y1,W2) + B2)
#copying from above

Y_ = tf.placeholder(tf.float32,[None,10])  #correct labels

In [39]:
#and now same thing as above
cross_entropy = - tf.reduce_sum(Y_ * tf.log(Y_out))

In [40]:
is_correct = tf.equal(tf.argmax(Y_out,1),tf.argmax(Y_,1)) #it will compute bool that is why cast is needed in the statement below
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))

In [41]:
#this time lets define decaying learning rate


initial_learning_rate = 0.003
global_step = tf.Variable(0, trainable=False)

learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step,
                                           5000, 0.96, staircase=True)

In [42]:
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy,global_step = global_step)

In [44]:
init = tf.global_variables_initializer()
sess.run(init)
for i in range(20001):
    batch_X,batch_Y = mnist.train.next_batch(100)
    train_data = {X : batch_X, Y_ : batch_Y}
    
    #training the system
    sess.run(train_step , feed_dict = train_data)
    
    #success
    a,c = sess.run([accuracy , cross_entropy],feed_dict= train_data)
    if i % 4000 == 0:
        print()
        print("Iteration number %d Training accuracy %f cross entropy loss %f"%(i,a,c))
        #success on test data
        
        test_data = {X : mnist.test.images,Y_ : mnist.test.labels}
        a,c = sess.run([accuracy,cross_entropy],feed_dict = test_data)
        
        print("Iteration number %d Testing accuracy %f cross entropy loss %f"%(i,a,c))



Iteration number 0 Training accuracy 0.160000 cross entropy loss 238.269150
Iteration number 0 Testing accuracy 0.112000 cross entropy loss 26001.328125

Iteration number 4000 Training accuracy 0.990000 cross entropy loss 6.191293
Iteration number 4000 Testing accuracy 0.949500 cross entropy loss 1673.652710

Iteration number 8000 Training accuracy 0.980000 cross entropy loss 5.255058
Iteration number 8000 Testing accuracy 0.963400 cross entropy loss 1193.652222

Iteration number 12000 Training accuracy 1.000000 cross entropy loss 5.236318
Iteration number 12000 Testing accuracy 0.971100 cross entropy loss 924.843689

Iteration number 16000 Training accuracy 1.000000 cross entropy loss 2.057953
Iteration number 16000 Testing accuracy 0.974400 cross entropy loss 824.327759

Iteration number 20000 Training accuracy 1.000000 cross entropy loss 2.411638
Iteration number 20000 Testing accuracy 0.976200 cross entropy loss 728.297241


### Same thing but this time will use relu

In [46]:
W1 = tf.Variable(tf.truncated_normal([28*28,300],stddev= 0.1))
B1 = tf.Variable(tf.ones([300])/10)

W2 = tf.Variable(tf.truncated_normal([300,10],stddev= 0.1))
B2 = tf.Variable(tf.ones([10])/10)

Y1 =  tf.nn.relu(tf.matmul(tf.reshape(X,[-1,784]),W1) + B1) #single line change
Y_out = tf.nn.softmax(tf.matmul(Y1,W2) + B2)
#copying from above

Y_ = tf.placeholder(tf.float32,[None,10])  #correct labels

#and now same thing as above
cross_entropy = - tf.reduce_sum(Y_ * tf.log(Y_out))

is_correct = tf.equal(tf.argmax(Y_out,1),tf.argmax(Y_,1)) #it will compute bool that is why cast is needed in the statement below
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))

#this time lets define decaying learning rate


initial_learning_rate = 0.003
global_step = tf.Variable(0, trainable=False)

learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step,
                                           5000, 0.96, staircase=True)

train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy,global_step = global_step)


init = tf.global_variables_initializer()
sess.run(init)
for i in range(20001):
    batch_X,batch_Y = mnist.train.next_batch(100)
    train_data = {X : batch_X, Y_ : batch_Y}
    
    #training the system
    sess.run(train_step , feed_dict = train_data)
    
    #success
    a,c = sess.run([accuracy , cross_entropy],feed_dict= train_data)
    if i % 4000 == 0:
        print()
        print("Iteration number %d Training accuracy %f cross entropy loss %f"%(i,a,c))
        #success on test data
        
        test_data = {X : mnist.test.images,Y_ : mnist.test.labels}
        a,c = sess.run([accuracy,cross_entropy],feed_dict = test_data)
        
        print("Iteration number %d Testing accuracy %f cross entropy loss %f"%(i,a,c))



Iteration number 0 Training accuracy 0.310000 cross entropy loss 252.920029
Iteration number 0 Testing accuracy 0.215100 cross entropy loss 30305.580078

Iteration number 4000 Training accuracy 1.000000 cross entropy loss 1.456518
Iteration number 4000 Testing accuracy 0.976000 cross entropy loss 775.170654

Iteration number 8000 Training accuracy 1.000000 cross entropy loss 0.380347
Iteration number 8000 Testing accuracy 0.979700 cross entropy loss 680.427673

Iteration number 12000 Training accuracy 1.000000 cross entropy loss 0.483505
Iteration number 12000 Testing accuracy 0.981500 cross entropy loss 650.204041

Iteration number 16000 Training accuracy 1.000000 cross entropy loss 0.282901
Iteration number 16000 Testing accuracy 0.981900 cross entropy loss 669.653931

Iteration number 20000 Training accuracy 1.000000 cross entropy loss 0.186634
Iteration number 20000 Testing accuracy 0.982000 cross entropy loss 670.175720


<pre>
Now that is kind of impressive 98.2 % accuracy 
</pre>

### This time we will use dropout and Adam optimizer

In [47]:
W1 = tf.Variable(tf.truncated_normal([28*28,300],stddev= 0.1))
B1 = tf.Variable(tf.ones([300])/10)

W2 = tf.Variable(tf.truncated_normal([300,10],stddev= 0.1))
B2 = tf.Variable(tf.ones([10])/10)

Y1 =  tf.nn.relu(tf.matmul(tf.reshape(X,[-1,784]),W1) + B1) #single line change
Y1 = tf.nn.dropout(Y1,keep_prob= 0.7)
Y_out = tf.nn.softmax(tf.matmul(Y1,W2) + B2)
#copying from above

Y_ = tf.placeholder(tf.float32,[None,10])  #correct labels

#and now same thing as above
cross_entropy = - tf.reduce_sum(Y_ * tf.log(Y_out))

is_correct = tf.equal(tf.argmax(Y_out,1),tf.argmax(Y_,1)) #it will compute bool that is why cast is needed in the statement below
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))

#this time lets define decaying learning rate


initial_learning_rate = 0.003
global_step = tf.Variable(0, trainable=False)

learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step,
                                           5000, 0.96, staircase=True)

train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy,global_step = global_step)


init = tf.global_variables_initializer()
sess.run(init)
for i in range(20001):
    batch_X,batch_Y = mnist.train.next_batch(100)
    train_data = {X : batch_X, Y_ : batch_Y}
    
    #training the system
    sess.run(train_step , feed_dict = train_data)
    
    #success
    a,c = sess.run([accuracy , cross_entropy],feed_dict= train_data)
    if i % 4000 == 0:
        print()
        print("Iteration number %d Training accuracy %f cross entropy loss %f"%(i,a,c))
        #success on test data
        
        test_data = {X : mnist.test.images,Y_ : mnist.test.labels}
        a,c = sess.run([accuracy,cross_entropy],feed_dict = test_data)
        
        print("Iteration number %d Testing accuracy %f cross entropy loss %f"%(i,a,c))



Iteration number 0 Training accuracy 0.380000 cross entropy loss 175.315506
Iteration number 0 Testing accuracy 0.248200 cross entropy loss 22249.929688

Iteration number 4000 Training accuracy 0.110000 cross entropy loss nan
Iteration number 4000 Testing accuracy 0.098000 cross entropy loss nan

Iteration number 8000 Training accuracy 0.130000 cross entropy loss nan
Iteration number 8000 Testing accuracy 0.098000 cross entropy loss nan

Iteration number 12000 Training accuracy 0.080000 cross entropy loss nan
Iteration number 12000 Testing accuracy 0.098000 cross entropy loss nan

Iteration number 16000 Training accuracy 0.120000 cross entropy loss nan
Iteration number 16000 Testing accuracy 0.098000 cross entropy loss nan

Iteration number 20000 Training accuracy 0.110000 cross entropy loss nan
Iteration number 20000 Testing accuracy 0.098000 cross entropy loss nan


### Code above gives loss as nan  solution use softmax_cross_entropy_with_logits

In [69]:
#copying from above
W1 = tf.Variable(tf.truncated_normal([28*28,300],stddev= 0.1))
B1 = tf.Variable(tf.ones([300])/10)

W2 = tf.Variable(tf.truncated_normal([300,10],stddev= 0.1))
B2 = tf.Variable(tf.ones([10])/10)

Y1 =  tf.nn.relu(tf.matmul(tf.reshape(X,[-1,784]),W1) + B1) #single line change
Y1 = tf.nn.dropout(Y1,keep_prob= 0.7)
#Y_out = tf.nn.softmax(tf.matmul(Y1,W2) + B2)

#we will only calculate logits softmax will be done together with cross entropy in a numericaly stable way
Y_logits = tf.matmul(Y1,W2) + B2

Y_ = tf.placeholder(tf.float32,[None,10])  #correct labels

#cross_entropy = - tf.reduce_sum(Y_ * tf.log(Y_out))
#a new function https://codelabs.developers.google.com/codelabs/cloud-tensorflow-mnist/#7

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = Y_logits,labels = Y_)
cross_entropy = tf.reduce_mean(cross_entropy) * 100 #as suggested in the tutorial 

is_correct = tf.equal(tf.argmax(Y_logits,1),tf.argmax(Y_,1)) #it will compute bool that is why cast is needed in the statement below
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))

#this time lets define decaying learning rate


initial_learning_rate = 0.003
global_step = tf.Variable(0, trainable=False)

learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step,
                                           5000, 0.96, staircase=True)


train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy,global_step = global_step)


init = tf.global_variables_initializer()
sess.run(init)
for i in range(20001):
    batch_X,batch_Y = mnist.train.next_batch(100)
    train_data = {X : batch_X, Y_ : batch_Y}
    
    #training the system
    sess.run(train_step , feed_dict = train_data)
    
    #success
    a,c = sess.run([accuracy , cross_entropy],feed_dict= train_data)
    
    if i % 4000 == 0:
        print()
        #print(c)
        print("Iteration number %d Training accuracy %f cross entropy loss %f"%(i,a,c))
        #print("Training iteration number",i,"accuracy",a,"loss",c)
        #success on test data
        
        test_data = {X : mnist.test.images,Y_ : mnist.test.labels}
        a,c = sess.run([accuracy,cross_entropy],feed_dict = test_data)
        
        #print("Training iteration number",i,"accuracy",a,"loss",c)
        print("Iteration number %d Testing accuracy %f cross entropy loss %f"%(i,a,c))



Iteration number 0 Training accuracy 0.320000 cross entropy loss 194.676025
Iteration number 0 Testing accuracy 0.202100 cross entropy loss 230.038956

Iteration number 4000 Training accuracy 1.000000 cross entropy loss 0.777944
Iteration number 4000 Testing accuracy 0.970600 cross entropy loss 10.565776

Iteration number 8000 Training accuracy 1.000000 cross entropy loss 0.280782
Iteration number 8000 Testing accuracy 0.971300 cross entropy loss 12.314648

Iteration number 12000 Training accuracy 0.990000 cross entropy loss 1.266635
Iteration number 12000 Testing accuracy 0.976300 cross entropy loss 13.126901

Iteration number 16000 Training accuracy 1.000000 cross entropy loss 0.159158
Iteration number 16000 Testing accuracy 0.976400 cross entropy loss 13.321301

Iteration number 20000 Training accuracy 1.000000 cross entropy loss 0.120493
Iteration number 20000 Testing accuracy 0.976600 cross entropy loss 15.962158


In [73]:
len(graph.get_operations())

5018

## Now to Convolutions 

In [20]:
#copying directly from the video
K = 4
L = 8
M = 12


#k number of filters
W1 = tf.Variable(tf.truncated_normal([5,5,1,K],stddev= 0.1))
B1 = tf.Variable(tf.ones([K])/10)


#k channels in previous output and L number of filters
W2 = tf.Variable(tf.truncated_normal([5,5,K,L],stddev= 0.1))
B2 = tf.Variable(tf.ones([L])/10)


W3 = tf.Variable(tf.truncated_normal([5,5,L,M],stddev= 0.1))
B3 = tf.Variable(tf.ones([M])/10)

#fully connected layer after this
N = 200

W4 = tf.Variable(tf.truncated_normal([7*7*M,N],stddev=0.1))
B4 = tf.Variable(tf.ones([N])/10)

W5 = tf.Variable(tf.truncated_normal([N,10],stddev= 0.1))
B5 = tf.Variable(tf.ones([10])/10)

In [21]:
len(graph.get_operations())

152

In [22]:
#here X is (100,28,28,1) 100 is #of images
Y1 = tf.nn.relu(tf.nn.conv2d(X, W1, strides = [1,1,1,1], padding= 'SAME') +B1) #output is (100,28,28,4)

Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides = [1,2,2,1], padding= 'SAME') +B2) #output is (100,14,14,8)

Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides = [1,2,2,1], padding= 'SAME') +B3) #output is (100,7,7,12)

YY = tf.reshape(Y3, shape = [-1, 7 * 7 * M])  #essentially it will be (100,7*7*12)

Y4 = tf.nn.relu(tf.matmul(YY,W4) + B4)                 #(100,200)
#Y = tf.nn.softmax(tf.matmul(Y4,W5) + B5)               #(100,10)

In [24]:
#we will do it with softmax with logits for numerical stability reasons
Y_logits = tf.matmul(Y4,W5) + B5
Y_ = tf.placeholder(tf.float32,[None,10])


cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = Y_logits,labels = Y_)
cross_entropy = tf.reduce_mean(cross_entropy) * 100 

is_correct = tf.equal(tf.argmax(Y_logits,1),tf.argmax(Y_,1)) #it will compute bool that is why cast is needed in the statement below
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))

#this time lets define decaying learning rate


initial_learning_rate = 0.003
global_step = tf.Variable(0, trainable=False)

learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step,
                                           5000, 0.96, staircase=True)


train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy,global_step = global_step)


init = tf.global_variables_initializer()
sess.run(init)
for i in range(4001):
    batch_X,batch_Y = mnist.train.next_batch(100)
    train_data = {X : batch_X, Y_ : batch_Y}
    
    #training the system
    sess.run(train_step , feed_dict = train_data)
    
    #success
    a,c = sess.run([accuracy , cross_entropy],feed_dict= train_data)
    
    if i % 500 == 0:
        print()
        #print(c)
        print("Iteration number %d Training accuracy %f cross entropy loss %f"%(i,a,c))
        #print("Training iteration number",i,"accuracy",a,"loss",c)
        #success on test data
        
        test_data = {X : mnist.test.images,Y_ : mnist.test.labels}
        a,c = sess.run([accuracy,cross_entropy],feed_dict = test_data)
        
        #print("Training iteration number",i,"accuracy",a,"loss",c)
        print("Iteration number %d Testing accuracy %f cross entropy loss %f"%(i,a,c))




Iteration number 0 Training accuracy 0.150000 cross entropy loss 225.750488
Iteration number 0 Testing accuracy 0.116300 cross entropy loss 237.257477

Iteration number 500 Training accuracy 0.960000 cross entropy loss 13.901351
Iteration number 500 Testing accuracy 0.974000 cross entropy loss 7.816738

Iteration number 1000 Training accuracy 1.000000 cross entropy loss 1.759503
Iteration number 1000 Testing accuracy 0.981300 cross entropy loss 5.586134

Iteration number 1500 Training accuracy 0.980000 cross entropy loss 3.804555
Iteration number 1500 Testing accuracy 0.982900 cross entropy loss 5.216575

Iteration number 2000 Training accuracy 0.990000 cross entropy loss 2.985325
Iteration number 2000 Testing accuracy 0.985900 cross entropy loss 4.586003

Iteration number 2500 Training accuracy 1.000000 cross entropy loss 0.114356
Iteration number 2500 Testing accuracy 0.986800 cross entropy loss 4.306031

Iteration number 3000 Training accuracy 1.000000 cross entropy loss 1.594681
I

### 98.72 that is kind of cool. Lets now try a different architecture and this time with dropout

In [28]:
#copying directly from the video. Giving more degrees of freedom to the system 
#i.e. increasing the number of filters or number of Ws
K = 6
L = 12
M = 24


#k number of filters
W1 = tf.Variable(tf.truncated_normal([6,6,1,K],stddev= 0.1))
B1 = tf.Variable(tf.ones([K])/10)


#k channels in previous output and L number of filters
W2 = tf.Variable(tf.truncated_normal([5,5,K,L],stddev= 0.1))
B2 = tf.Variable(tf.ones([L])/10)


W3 = tf.Variable(tf.truncated_normal([4,4,L,M],stddev= 0.1))
B3 = tf.Variable(tf.ones([M])/10)

#fully connected layer after this
N = 200

W4 = tf.Variable(tf.truncated_normal([7*7*M,N],stddev=0.1))
B4 = tf.Variable(tf.ones([N])/10)

W5 = tf.Variable(tf.truncated_normal([N,10],stddev= 0.1))
B5 = tf.Variable(tf.ones([10])/10)

In [29]:
len(graph.get_operations())

920

In [37]:
#here X is (100,28,28,1) 100 is #of images
Y1 = tf.nn.relu(tf.nn.conv2d(X, W1, strides = [1,1,1,1], padding= 'SAME') +B1) #output is (100,28,28,6)

Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides = [1,2,2,1], padding= 'SAME') +B2) #output is (100,14,14,12)

Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides = [1,2,2,1], padding= 'SAME') +B3) #output is (100,7,7,24)

YY = tf.reshape(Y3, shape = [-1, 7 * 7 * M])  #essentially it will be (100,7*7*24)

Y4 = tf.nn.relu(tf.matmul(YY,W4) + B4)                 #(100,200)
#Y = tf.nn.softmax(tf.matmul(Y4,W5) + B5)               #(100,10)

Y4 = tf.nn.dropout(Y4,keep_prob= 0.75)    #dropout introduced

#we will do it with softmax with logits for numerical stability reasons
Y_logits = tf.matmul(Y4,W5) + B5

In [40]:
Y_ = tf.placeholder(tf.float32,[None,10])


cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = Y_logits,labels = Y_)
cross_entropy = tf.reduce_mean(cross_entropy) * 100 

is_correct = tf.equal(tf.argmax(Y_logits,1),tf.argmax(Y_,1)) #it will compute bool that is why cast is needed in the statement below
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))

#this time lets define decaying learning rate


initial_learning_rate = 0.003
global_step = tf.Variable(0, trainable=False)

learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step,
                                           100, 0.96, staircase=True)


train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy,global_step = global_step)


init = tf.global_variables_initializer()
sess.run(init)
#lets run for 10000
for i in range(20001):
    batch_X,batch_Y = mnist.train.next_batch(100)
    train_data = {X : batch_X, Y_ : batch_Y}
    
    #training the system
    sess.run(train_step , feed_dict = train_data)
    
    #success
    a,c = sess.run([accuracy , cross_entropy],feed_dict= train_data)
    
    if i % 5000 == 0:
        print()
        #print(c)
        print("Iteration number %d Training accuracy %f cross entropy loss %f"%(i,a,c))
        #print("Training iteration number",i,"accuracy",a,"loss",c)
        #success on test data
        
        test_data = {X : mnist.test.images,Y_ : mnist.test.labels}
        a,c = sess.run([accuracy,cross_entropy],feed_dict = test_data)
        
        #print("Training iteration number",i,"accuracy",a,"loss",c)
        print("Iteration number %d Testing accuracy %f cross entropy loss %f"%(i,a,c))




Iteration number 0 Training accuracy 0.130000 cross entropy loss 221.796997
Iteration number 0 Testing accuracy 0.128100 cross entropy loss 231.528336

Iteration number 5000 Training accuracy 1.000000 cross entropy loss 0.035542
Iteration number 5000 Testing accuracy 0.989700 cross entropy loss 4.091547

Iteration number 10000 Training accuracy 1.000000 cross entropy loss 0.003339
Iteration number 10000 Testing accuracy 0.991500 cross entropy loss 4.168932

Iteration number 15000 Training accuracy 1.000000 cross entropy loss 0.035732
Iteration number 15000 Testing accuracy 0.991400 cross entropy loss 4.405719

Iteration number 20000 Training accuracy 1.000000 cross entropy loss 0.000704
Iteration number 20000 Testing accuracy 0.991500 cross entropy loss 4.513232


### Cool 99.15 % finally key is decreasing learning rate and more number of iterations

<pre> 
This was further improved to 99.5% by the speaker with batch normalization
</pre>